Ash Particle Size Distribution - Suitable for Health Stakeholders
--

Plot for all CSV files in this folder, gives plots a suitable file name

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import os

%matplotlib inline

In [ ]:
style.use('fivethirtyeight')

In [ ]:
figsize = (15,5)

for file in sorted(os.listdir('.')):
        if file.endswith('.csv'):
            stem = file[:-3]
            plotfile = stem+'png'          

            #get and process data
            #data
            names = ['diameter', 'volume %', 'minus2sd', 'plus2sd', 'cumulative']
            data = pd.read_csv(file, skiprows=13, names=names, index_col=0)
            data.dropna(inplace=True)

            #put diamater into mm
            data.index /= 1000

            #calculate percentages in size ranges
            #convert to string for plottting, with 1 decimal place
            pc_lung = '{:.1f}'.format(data['volume %'][data.index<0.004].sum())+'%'
            pc_upper = '{:.1f}'.format(data['volume %'][(data.index>=0.004)&(data.index<0.01)].sum())+'%'
            pc_throat = '{:.1f}'.format(data['volume %'][(data.index>=0.01)&(data.index<0.1)].sum())+'%'
            pc_unlikely = '{:.1f}'.format(data['volume %'][data.index>0.1].sum())+'%'

            #get and process data description
            #description of data
            names = ['category', 'value']
            desc = pd.read_csv(file, usecols=[0,1], index_col=0, names=names, nrows=7, skiprows=1)

            #text descriptors for title
            fileid = desc.value[desc.index=='File ID:'].iloc[0]
            sampleid = desc.value[desc.index=='Sample ID:'].iloc[0]
            comment1 = desc.value[desc.index=='Comment 1:'].iloc[0]
            comment2 = desc.value[desc.index=='Comment 2:'].iloc[0]

            title = fileid+' : '+sampleid+', '+comment1+' : '+comment2

            #plot curve with extra information
            #plot curve
            fig,ax = plt.subplots(1,figsize=figsize)
            ax.semilogx(data.index, data['volume %'])
            #with 2sd errors
            #ax.fill_between(data.index, data['volume %']-data['minus2sd'], data['volume %']+data['minus2sd'], color='lightblue', alpha=0.5)

            #x-axis labels as decimal with 3 significant digits, rather than scientific notation
            ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))

            #colour areas
            ax.axvspan(0, 0.004, alpha=0.6, color='red') #< 0.004 mm, deep lungs
            ax.axvspan(0.004, 0.01, alpha=0.4, color='red') #0.004-0.01 mm, trachea (windpipe) & bronchial tubes
            ax.axvspan(0.01, 0.1, alpha=0.2, color='red') #0.01-0.1 mm, nose & throat
            ax.axvspan(0.1, 2, alpha=0.1, color='red') #>0.1-0.1 mm, unlikely to be inhaled

            #text labelling coloured categories
            ypos = 0.7 * ax.get_ylim()[1] #70% to top of axis
            ax.text(0.001, ypos, 'deep lungs', style='italic', fontsize=15, horizontalalignment='center')
            ax.text(0.0065, ypos, 'trachea\n(windpipe)\n&\nbronchial\ntubes', style='italic', fontsize=15, horizontalalignment='center', verticalalignment='center')
            ax.text(0.03, ypos, 'nose & throat', style='italic', fontsize=15, horizontalalignment='center')
            ax.text(0.7, ypos, 'unlikely to be inhaled', style='italic', fontsize=15, horizontalalignment='center')

            #text labelling percentages in each category
            ypos = 0.50 * ax.get_ylim()[1] #50% to top of axis
            ax.text(0.001, ypos, pc_lung, style='italic', fontsize=15, horizontalalignment='center')
            ax.text(0.0065, ypos, pc_upper, style='italic', fontsize=15, horizontalalignment='center')
            ax.text(0.03, ypos, pc_throat, style='italic', fontsize=15, horizontalalignment='center')
            ax.text(0.7, ypos, pc_unlikely, style='italic', fontsize=15, horizontalalignment='center')

            #axis limits and labels
            ax.set_xlim(xmin=0.0003, xmax=2)
            ax.set_ylabel('% volume of ash')
            ax.set_xlabel('ash particle diameter (mm)')
            ax.set_title(title)
            
            plt.savefig(plotfile, bbox_inches = 'tight')
            plt.close